# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.

# Definindo a URL filtrada pelo RJ
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

#Convertendo em json
data_json = requests.get(url).json()

#Transformando em df
df = pd.DataFrame(data_json['items'])

In [3]:
# 2) Quais são os três status das solicitações mais frequentes na base? Quais são suas frequências?
df.status.value_counts().head(3)

status
Deferido        105
Arquivado        53
Regularizado     26
Name: count, dtype: int64

In [4]:
# 3) Construa uma nova variável que contenha o ano do status. Observe que data_status vem como tipo object no DataFrame. 
# Dica: você pode usar o método .str para transformar o tipo da variável em string, em seguida um método como slice() ou split().

df['ano_protocol'] = df['data_protocolo'].astype(str).str.slice(0, 4)

In [5]:
# 4. Indique a frequência de cada ano do campo construído no item (3).
df.ano_protocol.value_counts()

ano_protocol
2008    40
None    25
2007    25
2014    23
2023    19
2010    18
2012    15
2011    13
2020    12
2013    12
2022     9
2009     7
2006     7
2018     7
2002     5
2016     4
2019     4
2017     4
2021     4
2015     3
2005     2
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
# 1) Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado ('Estado'ou Município), 
# e que devolve os dados da consulta no formato DataFrame.

def consulta_estado(uf,interesse='estado'):
    if interesse == 'estado':
        url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf='+uf+'&tipo_interessado=Estado'
    elif interesse == 'municipio':
        url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf='+uf+'&tipo_interessado=Munic%C3%ADpio'
    data_json = requests.get(url).json()
    df = pd.DataFrame(data_json['items'])
    return df

In [7]:
# 2) Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?
(consulta_estado('MG','estado').status == 'Arquivado por decurso de prazo').sum()

1

In [8]:
# 3) Qual é o município da Bahia com mais solicitações deferidas?
consulta_estado('BA','municipio')[consulta_estado('BA','municipio')['status'] =='Deferido'].interessado.value_counts().idxmax()

'Luís Eduardo Magalhães'

In [9]:
# 4) Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'
consulta_estado('BA','estado').to_csv('consulta_BA_estado.csv')